## 序贯（Sequential）模型
序贯模型是多个网络层的线性堆叠。

Dense层是常用的全连接层，所实现的运算是`output = activation(dot(input, kernel)+bias)`。其中`activatio`n是逐元素计算的激活函数，`kernel`是本层的权值矩阵，`bias`为偏置向量，只有当`use_bias=True`才会添加。

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(input_shape=(32,), units=784),
    Activation('relu'),
    Dense(units=10),
    Activation('softmax')
])

也可以通过`.add()`方法一个个地将layer加入模型中：

In [7]:
model = Sequential()
model.add(Dense(units=32, input_shape=(784, )))
model.add(Activation('relu'))

### 指定输入数据的shape
Sequential的第一层需要接收一个关于输入模型shape的参数，后面各个层可以自动推导出中间数据的shape。有几种方法指定第一层输入数据的shape。

+ 传递一个`input_shape`关键字给第一层，`input_shape`是一个tuple型数据，其中可以填入**None**，表示此位置**可能是任何正整数**，数据的batch_size不应包含在其中。
+ 有些2D层，如Dense，支持通过指定其输入维度`input_dim`来隐含的指定输入数据shape。一些3D的时域层支持通过参数`input_dim`和`input_length`来指定输入shape。
+ 如果你需要为输入指定一个固定大小的`batch_size`（常用于stateful RNN网络），可以传递batch_size参数到一个层中，例如你想指定输入张量的batch大小是32，数据shape是（6，8），则你需要传递`batch_size=32`和`input_shape=(6,8)`。

### 编译
`compile`接收三个参数。
+ 优化器optimizer：可以传入预定义的优化器名，如`rmsprop`，`adagrad`。或一个`Optimizer`类的对象。
+ 损失函数loss
+ 指标列表metrics。对于分类问题，一般将该列表设置为`metrics=['accuracy']`

In [11]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# For a binary classification problem
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
             loss='mse')

# 自定义指标
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy', mean_pred])

### 训练
Keras以Numpy数组作为输入数据和标签的数据类型。训练模型一般使用fit函数，该函数的详情见这里。下面是一些例子。

In [15]:
# For a single-input model with 2 classes (binary classification):
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=100))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.7033 - acc: 0.4960
Epoch 2/10
1000/1000 [==============================] - 0s 33us/step - loss: 0.6946 - acc: 0.5190
Epoch 3/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6881 - acc: 0.5390
Epoch 4/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6872 - acc: 0.5320
Epoch 5/10
1000/1000 [==============================] - 0s 35us/step - loss: 0.6828 - acc: 0.5570
Epoch 6/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6785 - acc: 0.5670
Epoch 7/10
1000/1000 [==============================] - 0s 32us/step - loss: 0.6738 - acc: 0.5800
Epoch 8/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6753 - acc: 0.5710
Epoch 9/10
1000/1000 [==============================] - 0s 32us/step - loss: 0.6693 - acc: 0.5810
Epoch 10/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6680 - acc: 0.6010
